In [1]:
import pandas as pd
from astropy.io import fits
import numpy as np
import movie_stars
import warnings
warnings.filterwarnings("ignore")
df_apogee = pd.read_csv('/Users/cam/Downloads/apogee_dr17.csv')
df_sos = pd.read_csv('/Users/cam/Downloads/Tsantaki_SoS_291021_CDS.csv')


In [4]:
df_open_clusters = pd.read_csv('/Users/cam/Downloads/open_clusters_3d.csv')
df_oc_full = pd.read_csv('/Users/cam/Downloads/open_clusters_full.csv')


df_oc_full = pd.merge(left=df_oc_full, right=df_apogee, how='left',
                      left_on='source_id', right_on='gaiaedr3_source_id')
df_oc_full = pd.merge(left=df_oc_full, right=df_sos,
                      how='left', left_on='source_id', right_on='gaiaSourceId')


In [5]:
df_oc_apogee = df_oc_full.loc[df_oc_full['vhelio_avg'].notnull()]
df_oc_apogee['rv_chosen'] = df_oc_apogee['vhelio_avg']

df_oc_sos = df_oc_full.loc[(df_oc_full['RVcor_merged'].notnull()) & (
    df_oc_full['vhelio_avg'].isnull())]
df_oc_sos['rv_chosen'] = df_oc_sos['RVcor_merged']

df_oc_gaia = df_oc_full.loc[(df_oc_full['rv'].notnull()) & (
    df_oc_full['vhelio_avg'].isnull()) & (df_oc_full['RVcor_merged'].isnull())]
df_oc_gaia['rv_chosen'] = df_oc_gaia['rv']

df_oc_no_rv = df_oc_full.loc[(df_oc_full['rv'].isnull()) & (
    df_oc_full['vhelio_avg'].isnull()) & (df_oc_full['RVcor_merged'].isnull())]

df_oc_full = pd.concat([df_oc_apogee, df_oc_sos, df_oc_gaia, df_oc_no_rv])


In [6]:
oc_grouped_df_list = []
oc_groups = df_oc_full.groupby('cluster')
for cluster, df_oc_grouped in oc_groups:
    df_oc_grouped = df_oc_grouped.rename(columns={'ra_x': 'ra_med',
                                                  'dec': 'dec_med',
                                                  'parallax': 'parallax_med',
                                                  'pmra': 'pmra_med',
                                                  'pmdec': 'pmdec_med',
                                                  'rv_chosen': 'rv_chosen_med'
                                                  })
    df_oc_grouped = df_oc_grouped.loc[df_oc_grouped['rv_chosen_med'].notnull()]
    # if len(df_oc_grouped) > 5:
    #df_oc_grouped = df_oc_grouped.median()
    df_oc_grouped['cluster'] = cluster
    df_oc_grouped['n'] = len(df_oc_grouped)
    df_oc_grouped['n_rv'] = len(
        df_oc_grouped.loc[df_oc_grouped.rv_chosen_med.notnull()])
    oc_grouped_df_list.append(df_oc_grouped)

df_oc_grouped_full = pd.concat(oc_grouped_df_list)
df_oc_grouped_full = df_oc_grouped_full.groupby(
    'cluster').median().reset_index()

df_open_clusters = pd.merge(left=df_open_clusters, right=df_oc_grouped_full,
                            how='left', left_on='cluster', right_on='cluster')
df_open_clusters.drop(['ra', 'dec', 'pmra', 'pmdec',
                      'parallax'], axis=1, inplace=True)

df_open_clusters.to_csv('/Users/cam/Downloads/cg_ocs_full.csv', index=False)


In [7]:
# crossmatch Sebastian catalog
df_sc = pd.read_csv('/Users/cam/Downloads/sco_cen_full.csv')
filter_col = [col for col in df_sc if col.startswith('cluster')]
sc_clusters_list = []
for c in filter_col:
    df_in_cluster = df_sc.loc[df_sc[c] == True]
    df_in_cluster['label'] = c
    sc_clusters_list.append(df_in_cluster)
df_sc = pd.concat(sc_clusters_list)

df_sc_apogee_xmatch = pd.merge(left=df_sc, right=df_apogee, how='left',
                               left_on='source_id', right_on='gaiaedr3_source_id', suffixes=('', '_y'))
df_sc_full = pd.merge(left=df_sc_apogee_xmatch, right=df_sos, how='left',
                      left_on='source_id', right_on='gaiaSourceId', suffixes=('', '_y'))


In [8]:
df_sc_apogee = df_sc_full.loc[df_sc_full['vhelio_avg'].notnull()]
df_sc_apogee['rv_chosen'] = df_sc_apogee['vhelio_avg']

df_sc_sos = df_sc_full.loc[(df_sc_full['RVcor_merged'].notnull()) & (
    df_sc_full['vhelio_avg'].isnull())]
df_sc_sos['rv_chosen'] = df_sc_sos['RVcor_merged']

df_sc_gaia = df_sc_full.loc[(df_sc_full['dr2_radial_velocity'].notnull()) & (
    df_sc_full['vhelio_avg'].isnull()) & (df_sc_full['RVcor_merged'].isnull())]
df_sc_gaia['rv_chosen'] = df_sc_gaia['dr2_radial_velocity']

df_sc_no_rv = df_sc_full.loc[(df_sc_full['dr2_radial_velocity'].isnull()) & (
    df_sc_full['vhelio_avg'].isnull()) & (df_sc_full['RVcor_merged'].isnull())]

df_sc_full = pd.concat([df_sc_apogee, df_sc_sos, df_sc_gaia, df_sc_no_rv])


In [9]:
sc_grouped_df_list = []
sc_groups = df_sc_full.groupby('label')
for cluster, df_sc_grouped in sc_groups:
    df_sc_grouped = df_sc_grouped.rename(columns={'ra_x': 'ra'})
    df_sc_grouped = df_sc_grouped.loc[df_sc_grouped['rv_chosen'].notnull()]
    if len(df_sc_grouped) > 20:
        #df_oc_grouped = df_oc_grouped.median()
        df_sc_grouped['label'] = cluster
        df_sc_grouped['n'] = len(df_sc_grouped)
        df_sc_grouped['n_rv'] = len(df_sc_grouped['rv_chosen'].notnull())
        sc_grouped_df_list.append(df_sc_grouped)

df_sc_grouped_full = pd.concat(sc_grouped_df_list)
df_sc_grouped_full = df_sc_grouped_full.groupby('label').median().reset_index()

#df_sc_out = pd.merge(left = df_sc_full, right = df_sc_grouped_full, how = 'left', left_on = 'cluster', right_on = 'cluster')
#df_sc_out.drop(['ra', 'dec', 'pmra', 'pmdec', 'parallax'], axis = 1, inplace=True)

df_sc_grouped_full.to_csv(
    '/Users/cam/Downloads/sc_sebastian_full.csv', index=False)


In [3]:
hdu_hao_full = fits.open('/Users/cam/Downloads/asu-34.fit')
d = hdu_hao_full[1].data
df_hao_full = pd.DataFrame({'ra': d['RA_ICRS'].byteswap().newbyteorder(),
                            'dec': d['DE_ICRS'].byteswap().newbyteorder(),
                            'parallax': d['plx'].byteswap().newbyteorder(),
                            'pmra': d['pmRA'].byteswap().newbyteorder(),
                            'pmdec': d['pmDE'].byteswap().newbyteorder(),
                            'radial_velocity': d['RV'].byteswap().newbyteorder(),
                            'Cluster': d['Cluster'].byteswap().newbyteorder(),
                            'source_id': d['GaiaEDR3'].byteswap().newbyteorder(),
                            'Gmag': d['Gmag'].byteswap().newbyteorder(),
                            'bp_rp': d['BP-RP'].byteswap().newbyteorder()
                            })

df_hao_apogee = pd.merge(left=df_hao_full, right=df_apogee, how='left',
                         left_on='source_id', right_on='gaiaedr3_source_id', suffixes=('', '_y'))
df_hao_full = pd.merge(left=df_hao_apogee, right=df_sos, how='left',
                       left_on='source_id', right_on='gaiaSourceId', suffixes=('', '_y'))


In [4]:
df_hao_apogee = df_hao_full.loc[df_hao_full['vhelio_avg'].notnull()]
df_hao_apogee['rv_chosen'] = df_hao_apogee['vhelio_avg']

df_hao_sos = df_hao_full.loc[(df_hao_full['RVcor_merged'].notnull()) & (
    df_hao_full['vhelio_avg'].isnull())]
df_hao_sos['rv_chosen'] = df_hao_sos['RVcor_merged']

df_hao_gaia = df_hao_full.loc[(df_hao_full['radial_velocity'].notnull()) & (
    df_hao_full['vhelio_avg'].isnull()) & (df_hao_full['RVcor_merged'].isnull())]
df_hao_gaia['rv_chosen'] = df_hao_gaia['radial_velocity']

df_hao_no_rv = df_hao_full.loc[(df_hao_full['radial_velocity'].isnull()) & (
    df_hao_full['vhelio_avg'].isnull()) & (df_hao_full['RVcor_merged'].isnull())]
df_hao_no_rv['rv_chosen'] = [np.nan]*len(df_hao_no_rv)

df_hao_full = pd.concat([df_hao_apogee, df_hao_sos, df_hao_gaia, df_hao_no_rv])
df_hao_full
df_hao_grouped = pd.read_csv('/Users/cam/Downloads/hao_grouped.csv')
clusters = df_hao_full.groupby(['Cluster'])

clusters_list = []
for c, df_c in clusters:
    df_c['age'] = df_hao_grouped.loc[df_hao_grouped.Cluster ==
                                     c].age.values*np.ones(len(df_c))
    df_c['n_rv'] = len(df_c.loc[df_c.rv_chosen.notnull()])*np.ones(len(df_c))
    clusters_list.append(df_c)
df_hao_full = pd.concat(clusters_list)


df_hao_full.to_csv('/Users/cam/Downloads/hao_oc_rv_full.csv', index=False)


,ra,dec,parallax,pmra,pmdec,radial_velocity,Cluster,source_id,Gmag,bp_rp,...,decl,photGMeanMag,RVcor_merged,ERRVcor_merged,ERRVcor_merged_sigma,surveysid,nameid,sosID,rv_chosen,age
0,267.999260,-27.873457,0.341980,0.732284,0.082886,NaN,OC-0001,4063468700560940500,14.324421,1.296638,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.118864
1,268.013597,-27.864480,0.357414,0.748455,0.201602,NaN,OC-0001,4063468734920671700,15.070120,1.308202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.118864
2,267.980555,-27.868026,0.346944,0.778789,0.141155,NaN,OC-0001,4063468769280455700,14.246620,1.217891,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.118864
3,268.025863,-27.858703,0.356996,0.795389,0.132624,NaN,OC-0001,4063468837999877100,16.092915,1.697772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.118864
4,268.026055,-27.843949,0.351122,0.617624,0.091736,NaN,OC-0001,4063468941079375900,15.568107,1.393621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.118864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19422,266.647611,-29.089829,0.508649,1.475444,-1.362072,NaN,OC-0704,4057470967637140500,17.430216,1.811058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.892541
19423,266.668342,-29.077437,0.504776,1.606914,-0.955104,NaN,OC-0704,4057471036356613600,14.278894,1.288549,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.892541
19424,266.616146,-29.088442,0.517978,1.828581,-0.658679,NaN,OC-0704,4057471689191664600,16.687235,1.705700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.892541
19425,266.611088,-29.237743,0.504744,1.861989,-0.817591,NaN,OC-0704,4057276074904741400,16.721592,1.547017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.892541


In [19]:
hdu_he_full = fits.open('/Users/cam/Downloads/He_OCs_full.fit')
d = hdu_he_full[1].data
df_he_full = pd.DataFrame(d)


df_he_apogee = pd.merge(left=df_he_full, right=df_apogee, how='left',
                        left_on='GaiaEDR3', right_on='gaiaedr3_source_id', suffixes=('', '_y'))
df_he_full = pd.merge(left=df_he_apogee, right=df_sos, how='left',
                      left_on='GaiaEDR3', right_on='gaiaSourceId', suffixes=('', '_y'))


In [36]:

df_he_apogee = df_he_full.loc[df_he_full['vhelio_avg'].notnull()]
df_he_apogee['rv_chosen'] = df_he_apogee['vhelio_avg']

df_he_sos = df_he_full.loc[(df_he_full['RVcor_merged'].notnull()) & (
    df_he_full['vhelio_avg'].isnull())]
df_he_sos['rv_chosen'] = df_he_sos['RVcor_merged']

df_he_gaia = df_he_full.loc[(df_he_full['RV'] != 0.) & (
    df_he_full['vhelio_avg'].isnull()) & (df_he_full['RVcor_merged'].isnull())]
df_he_gaia['rv_chosen'] = df_he_gaia['RV']

df_he_no_rv = df_he_full.loc[(df_he_full['RV'] == 0.) & (
    df_he_full['vhelio_avg'].isnull()) & (df_he_full['RVcor_merged'].isnull())]
df_he_no_rv['rv_chosen'] = [np.nan]*len(df_he_no_rv)

df_he_full = pd.concat([df_he_apogee, df_he_sos, df_he_gaia, df_he_no_rv])

hdu_he_grouped = fits.open('/Users/cam/Downloads/He_OCs.fit')
df_he_grouped = pd.DataFrame(hdu_he_grouped[1].data)
clusters = df_he_full.groupby(['Cluster'])

clusters_list = []
for c, df_c in clusters:
    df_c['n'] = np.ones(len(df_c))*len(df_c)
    df_c['n_rv'] = len(df_c.loc[df_c['rv_chosen'].notnull()]
                       )*np.ones(len(df_c))
    df_c['age'] = (10**(df_he_grouped.loc[df_he_grouped.Cluster == c]
                   ['logAge'].values)/1e6)*np.ones(len(df_c))
    clusters_list.append(df_c)
df_he_full = pd.concat(clusters_list)


df_he_full.to_csv('/Users/cam/Downloads/he_oc_rv_full.csv', index=False)


In [18]:
# sebastian sco-cen new
sc_new_hdu = fits.open('/Users/cam/Downloads/2022-03-04-ScoCen-12550.fits')
df_sc_new = pd.DataFrame(sc_new_hdu[1].data)

filter_col = [col for col in df_sc_new if col.startswith('cluster')]
sc_clusters_list = []
for c in filter_col:
    df_in_cluster = df_sc_new.loc[df_sc_new[c] == True]
    df_in_cluster['label'] = c
    sc_clusters_list.append(df_in_cluster)
df_sc_new = pd.concat(sc_clusters_list)

df_sc_apogee_xmatch_new = pd.merge(left=df_sc_new, right=df_apogee, how='left',
                               left_on='dr3_source_id', right_on='gaiaedr3_source_id', suffixes=('', '_y'))
df_sc_full_new = pd.merge(left=df_sc_apogee_xmatch_new, right=df_sos, how='left',
                      left_on='dr3_source_id', right_on='gaiaSourceId', suffixes=('', '_y'))
df_sc_new.keys()

Index(['dr3_source_id', 'designation_dr3', 'dr2_source_id', 'cluster_47',
       'cluster_49', 'cluster_0', 'cluster_1', 'cluster_2', 'cluster_4',
       'cluster_6',
       ...
       'U', 'V', 'W', 'cluster_label', 'name_label', 'rag_label',
       'sample_type', 'cluster_age', 'scocen_reg', 'label'],
      dtype='object', length=456)

In [19]:
df_sc_apogee_new = df_sc_full_new.loc[df_sc_full_new['vhelio_avg'].notnull()]
df_sc_apogee_new['rv_chosen'] = df_sc_apogee_new['vhelio_avg']

df_sc_sos_new = df_sc_full_new.loc[(df_sc_full_new['RVcor_merged'].notnull()) & (
    df_sc_full_new['vhelio_avg'].isnull())]
df_sc_sos_new['rv_chosen'] = df_sc_sos_new['RVcor_merged']

df_sc_gaia_new = df_sc_full_new.loc[(df_sc_full_new['dr2_radial_velocity'].notnull()) & (
    df_sc_full_new['vhelio_avg'].isnull()) & (df_sc_full_new['RVcor_merged'].isnull())]
df_sc_gaia_new['rv_chosen'] = df_sc_gaia_new['dr2_radial_velocity']

df_sc_no_rv_new = df_sc_full_new.loc[(df_sc_full_new['dr2_radial_velocity'].isnull()) & (
    df_sc_full_new['vhelio_avg'].isnull()) & (df_sc_full_new['RVcor_merged'].isnull())]

df_sc_full_new = pd.concat([df_sc_apogee_new, df_sc_sos_new, df_sc_gaia_new, df_sc_no_rv_new])

In [20]:
sc_grouped_df_list_new = []
sc_groups_new = df_sc_full_new.groupby('label')
for cluster, df_sc_grouped_new in sc_groups_new:
    df_sc_grouped_new = df_sc_grouped_new.rename(columns={'ra_x': 'ra'})
    df_sc_grouped_new = df_sc_grouped_new.loc[df_sc_grouped_new['rv_chosen'].notnull()]
    if len(df_sc_grouped_new) > 20:
        #df_oc_grouped = df_oc_grouped.median()
        df_sc_grouped_new['label'] = cluster
        df_sc_grouped_new['n'] = len(df_sc_grouped_new)
        df_sc_grouped_new['n_rv'] = len(df_sc_grouped_new['rv_chosen'].notnull())
        sc_grouped_df_list_new.append(df_sc_grouped_new)

df_sc_grouped_full_new = pd.concat(sc_grouped_df_list_new)
df_sc_grouped_full_new = df_sc_grouped_full_new.groupby('label').median().reset_index()

#df_sc_out = pd.merge(left = df_sc_full, right = df_sc_grouped_full, how = 'left', left_on = 'cluster', right_on = 'cluster')
#df_sc_out.drop(['ra', 'dec', 'pmra', 'pmdec', 'parallax'], axis = 1, inplace=True)

df_sc_grouped_full_new.to_csv(
    '/Users/cam/Downloads/sc_sebastian_full_new.csv', index=False)

In [25]:
df_sc_grouped_full_new.dec

0    -24.607270
1    -35.331356
2    -39.684545
3    -24.832903
4    -23.493538
5    -34.556339
6    -39.055367
7    -63.477758
8    -54.193919
9    -58.353950
10   -64.396388
11   -43.174881
12   -50.721850
13   -67.854717
14   -24.339452
15   -24.450120
16   -24.413472
17   -22.451588
18   -19.647605
19   -19.427088
20   -20.633856
21   -22.489070
22   -48.495471
23   -76.593686
24   -36.924988
Name: dec, dtype: float64